In [1]:
import pandas as pd

# Extract
We use  csv  file to retrieve the Uber dataset which is saved locally.
The data is then loaded into a Pandas DataFrame for further processing.

In [4]:
file_path = "uber_data.csv"
df = pd.read_csv(file_path)

# Transformation

In [5]:
## Convert datetime columns to datetime type

df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])

## Remove duplicates and reset index
df = df.drop_duplicates().reset_index(drop=True)

## Assign unique trip ID
df['trip_id'] = df.index

## Data Modeling

### Dimension Tables
For better data normalization, we extract and structure the following:
1. **Date Time Dimension** (datetime_dim)
2. **Passenger Count Dimension** ( passenger_count_dim )
3. **Trip Distance Dimension** ( trip_distance_dim )
4. **Pickup & Dropoff Location Dimensions** ( pickup_location_dim ,  dropoff_location_dim )
5. **Rate Code Dimension** ( rate_code_dim ) – Mapped to readable descriptions.
6. **Payment Type Dimension** ( payment_type_dim ) – Translates IDs to names.


### 1.DateTime Dimension
We extract various time-related attributes from  tpep_pickup_datetime  and  tpep_dropoff_datetime  to create a separate **datetime dimension table**.

#### Attributes:
- Pickup & Dropoff: Hour, Day, Weekday, Month, Year
- Unique  datetime_id  assigned


In [6]:
# Create datetime_dim 
datetime_dim = df[['tpep_pickup_datetime', 'tpep_dropoff_datetime']].copy()
datetime_dim = datetime_dim.assign(
    pickup_hour=datetime_dim['tpep_pickup_datetime'].dt.hour,
    pickup_day=datetime_dim['tpep_pickup_datetime'].dt.day,
    pickup_weekday=datetime_dim['tpep_pickup_datetime'].dt.weekday,
    pickup_month=datetime_dim['tpep_pickup_datetime'].dt.month,
    pickup_year=datetime_dim['tpep_pickup_datetime'].dt.year,
    dropoff_hour=datetime_dim['tpep_dropoff_datetime'].dt.hour,
    dropoff_day=datetime_dim['tpep_dropoff_datetime'].dt.day,
    dropoff_weekday=datetime_dim['tpep_dropoff_datetime'].dt.weekday,
    dropoff_month=datetime_dim['tpep_dropoff_datetime'].dt.month,
    dropoff_year=datetime_dim['tpep_dropoff_datetime'].dt.year
).reset_index(drop=True)
datetime_dim.insert(0, 'datetime_id', datetime_dim.index)

### 2. Passenger Count Dimension

In [7]:
# Passenger Count Dimension
passenger_count_dim = df[['passenger_count']].copy()
passenger_count_dim.insert(0, 'passenger_count_id', passenger_count_dim.index)

### 3.Trip Distance Dimension

In [8]:
# Trip Distance Dimension
trip_distance_dim = df[['trip_distance']].copy()
trip_distance_dim.insert(0, 'trip_distance_id', trip_distance_dim.index)

### 4.Pickup Location Dimension

In [9]:
# Pickup Location Dimension
pickup_location_dim = df[['pickup_longitude', 'pickup_latitude']].copy()
pickup_location_dim.insert(0, 'pickup_location_id', pickup_location_dim.index)

### 5.Dropoff Location Dimension

In [10]:
# Dropoff Location Dimension
dropoff_location_dim = df[['dropoff_longitude', 'dropoff_latitude']].copy()
dropoff_location_dim.insert(0, 'dropoff_location_id', dropoff_location_dim.index)

### 6.Rate Code Dimension

In [11]:
# Rate Code Dimension
rate_code_type = {
    1: 'Standard rate', 2: 'JFK', 3: 'Newark', 4: 'Nassau or Westchester',
    5: 'Negotiated fare', 6: 'Group ride'
}
rate_code_dim = df[['RatecodeID']].copy()
rate_code_dim.insert(0, 'rate_code_id', rate_code_dim.index)
rate_code_dim['rate_code_name'] = rate_code_dim['RatecodeID'].map(rate_code_type)

### 7.Payment Type Dimension

In [12]:
# Payment Type Dimension
payment_type_name = {
    1: 'Credit card', 2: 'Cash', 3: 'No charge',
    4: 'Dispute', 5: 'Unknown', 6: 'Voided trip'
}
payment_type_dim = df[['payment_type']].copy()
payment_type_dim.insert(0, 'payment_type_id', payment_type_dim.index)
payment_type_dim['payment_type_name'] = payment_type_dim['payment_type'].map(payment_type_name)

### Fact Table
The **fact table** combines all relevant trip details, merging with dimension tables via unique keys.

#### Attributes:
-  trip_id ,  VendorID ,  fare_amount ,  extra ,  mta_tax ,  tip_amount ,  tolls_amount ,  improvement_surcharge ,  total_amount 
- Foreign keys linking to dimension tables.


In [13]:
fact_table = df.merge(passenger_count_dim, left_on='trip_id', right_on='passenger_count_id') \
                .merge(trip_distance_dim, left_on='trip_id', right_on='trip_distance_id') \
                .merge(rate_code_dim, left_on='trip_id', right_on='rate_code_id') \
                .merge(payment_type_dim, left_on='trip_id', right_on='payment_type_id') \
                .merge(datetime_dim, left_on='trip_id', right_on='datetime_id') \
                .merge(pickup_location_dim, left_on='trip_id', right_on='pickup_location_id') \
                .merge(dropoff_location_dim, left_on='trip_id', right_on='dropoff_location_id') \
                [['trip_id', 'VendorID', 'datetime_id', 'passenger_count_id', 'trip_distance_id',
                  'pickup_location_id', 'dropoff_location_id', 'rate_code_id', 'payment_type_id',
                  'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount',
                  'improvement_surcharge', 'total_amount']]

In [14]:
fact_table.head()

,trip_id,VendorID,datetime_id,passenger_count_id,trip_distance_id,pickup_location_id,dropoff_location_id,rate_code_id,payment_type_id,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,0,1,0,0,0,0,0,0,0,9.0,0.5,0.5,2.05,0.00,0.3,12.35
1,1,1,1,1,1,1,1,1,1,11.0,0.5,0.5,3.05,0.00,0.3,15.35
2,2,2,2,2,2,2,2,2,2,54.5,0.5,0.5,8.00,0.00,0.3,63.80
3,3,2,3,3,3,3,3,3,3,31.5,0.0,0.5,3.78,5.54,0.3,41.62
4,4,2,4,4,4,4,4,4,4,98.0,0.0,0.0,0.00,15.50,0.3,113.80


# Load

### Load dimension and fact dimensional dataframes as tables into postgresql

#### Connect to Postgres

In [16]:

from sqlalchemy import create_engine

# Database connection details
DB_HOST = "localhost"  # Or your Docker container IP
DB_PORT = "5432"
DB_NAME = "uber_data"
DB_USER = "admin"
DB_PASSWORD = "admin"

# Create a connection to PostgreSQL
engine = create_engine(f'postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}')
conn = engine.connect()


#### Insert DataFrames 

In [ ]:
# Optimize  .to_sql()  for large tables
def upload_dataframe(df, table_name):
    df.to_sql(
        table_name, 
        engine, 
        if_exists="replace", 
        index=False, 
        method="multi",  # BATCH INSERTS (Much Faster)
        chunksize=10_000  # Upload in chunks of 10K rows
    )
    print(f"✅ Uploaded {len(df)} rows into {table_name}")



In [18]:
# Upload all dfs as tables
upload_dataframe(datetime_dim, "datetime_dim")
upload_dataframe(passenger_count_dim, "passenger_count_dim")
upload_dataframe(trip_distance_dim, "trip_distance_dim")
upload_dataframe(pickup_location_dim, "pickup_location_dim")
upload_dataframe(dropoff_location_dim, "dropoff_location_dim")
upload_dataframe(rate_code_dim, "rate_code_dim")
upload_dataframe(payment_type_dim, "payment_type_dim")
upload_dataframe(fact_table, "fact_table")

✅ Uploaded 100000 rows into datetime_dim
✅ Uploaded 100000 rows into passenger_count_dim
✅ Uploaded 100000 rows into trip_distance_dim
✅ Uploaded 100000 rows into pickup_location_dim
✅ Uploaded 100000 rows into dropoff_location_dim
✅ Uploaded 100000 rows into rate_code_dim
✅ Uploaded 100000 rows into payment_type_dim
✅ Uploaded 100000 rows into fact_table


In [19]:
# Close connection
conn.close()
engine.dispose()

# Anyalysis 

In [21]:
from sqlalchemy import text
# Database connection details
DB_HOST = "localhost"  # Or your Docker container IP
DB_PORT = "5432"
DB_NAME = "uber_data"
DB_USER = "admin"
DB_PASSWORD = "admin"

# Create a connection to PostgreSQL
engine = create_engine(f'postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}')


# Define a function to execute the query and return results as a DataFrame
def run_query(query: str):
    with engine.connect() as connection:
        result = connection.execute(text(query))
        return pd.DataFrame(result.fetchall(), columns=result.keys())




#### Create/Replace the analytics table

In [ ]:
# First SQL Query: Create/Replace the analytics table

# Query to drop the table if it exists
drop_table_query = """
DROP TABLE IF EXISTS uber_analytics.analytics_table;
"""

# Query to create the new table
create_table_query = """
CREATE TABLE uber_analytics."analytics_table" AS
SELECT 
  f."trip_id",
  f."VendorID",
  d."tpep_pickup_datetime",
  d."tpep_dropoff_datetime",
  p."passenger_count",
  t."trip_distance",
  r."rate_code_name",
  pick."pickup_latitude",
  pick."pickup_longitude",
  drop."dropoff_latitude",
  drop."dropoff_longitude",
  pay."payment_type_name",
  f."fare_amount",
  f."extra",
  f."mta_tax",
  f."tip_amount",
  f."tolls_amount",
  f."improvement_surcharge",
  f."total_amount"
FROM public."fact_table" AS f
INNER JOIN public."datetime_dim" AS d  
  ON f."datetime_id" = d."datetime_id"
INNER JOIN public."passenger_count_dim" AS p
  ON p."passenger_count_id" = f."passenger_count_id"  
INNER JOIN public."trip_distance_dim" AS t
  ON t."trip_distance_id" = f."trip_distance_id"  
INNER JOIN public."rate_code_dim" AS r 
  ON r."rate_code_id" = f."rate_code_id"  
INNER JOIN public."pickup_location_dim" AS pick
  ON pick."pickup_location_id" = f."pickup_location_id"
INNER JOIN public."dropoff_location_dim" AS drop
  ON drop."dropoff_location_id" = f."dropoff_location_id"
INNER JOIN public."payment_type_dim" AS pay
  ON pay."payment_type_id" = f."payment_type_id";
"""


In [ ]:
# Execute the queries
with engine.connect() as connection:
    connection.execute(text(drop_table_query))  # Drop the table if it exists
    connection.execute(text(create_table_query))  # Create the table
    result = connection.execute(text(create_table_query))


print("Table created successfully in the uber_analytics schema!")

Table created successfully in the uber_analytics schema!


#### Fetch top 10 pickup locations 

In [47]:
# Second Query: Fetch top 10 pickup locations
pickup_query = """
SELECT 
  pickup_latitude, 
  pickup_longitude, 
  COUNT(trip_id) AS trip_count
FROM  uber_analytics.analytics_table 
GROUP BY pickup_latitude, pickup_longitude
ORDER BY trip_count DESC
LIMIT 10;
"""

pickup_results = run_query(pickup_query)
print(pickup_results)

   pickup_latitude  pickup_longitude  trip_count
0         0.000000          0.000000         925
1        40.747292        -73.938316          21
2        40.762550        -73.986588          20
3        40.715248        -73.755020          20
4        40.681492        -74.105141          20
5        40.812801        -73.960541          14
6        40.745331        -73.970848          13
7        40.751541        -73.945274          11
8        40.833084        -73.945747           9
9        40.784790        -73.958015           8


#### Get trips by passenger count

In [49]:
# Third Query: Get trips by passenger count
passenger_count_query = """
SELECT 
  p.passenger_count, 
  COUNT(f.trip_id) AS trip_count
FROM  public.fact_table  AS f
JOIN  public.passenger_count_dim  AS p
  ON f.passenger_count_id = p.passenger_count_id
GROUP BY p.passenger_count
ORDER BY trip_count DESC;
"""

passenger_count_results = run_query(passenger_count_query)
print(passenger_count_results)

   passenger_count  trip_count
0                1       65493
1                2       13709
2                5        8748
3                6        6077
4                3        4076
5                4        1894
6                0           3


#### Get average fare per pickup hour

In [51]:
# Fourth Query: Get average fare per pickup hour
fare_query = """
SELECT 
  d.pickup_hour, 
  AVG(f.fare_amount) AS avg_fare
FROM public.fact_table AS f
LEFT JOIN public.datetime_dim AS d
  ON f.datetime_id = d.datetime_id
GROUP BY d.pickup_hour
ORDER BY d.pickup_hour ASC;
"""

fare_results = run_query(fare_query)
print(fare_results)

    pickup_hour   avg_fare
0             0  14.008186
1             1  13.907300
2             2  12.901618
3             3  14.090516
4             4  16.071229
5             5  14.440222
6             6  13.584520
7             7  12.006951
8             8  12.213717
9             9  12.594821
10           10  12.973568
11           11  13.124655
12           12  13.243506
13           13  14.002043
14           14  14.959385


In [52]:
# Close connection
conn.close()
engine.dispose()